#Clone the Standard Github Avatar Repo

In [ ]:
!git clone https://github.com/xrtlab/Validated-Avatar-Library-for-Inclusion-and-Diversity---VALID.git
!cd Validated-Avatar-Library-for-Inclusion-and-Diversity---VALID/Avatars

In [ ]:
import os

base_raw_url = 'https://github.com/xrtlab/Validated-Avatar-Library-for-Inclusion-and-Diversity---VALID/raw/refs/heads/main/Avatars'
raws = []

for root, dirs, files in os.walk('.'):
    for file in files:
        if file.endswith('.fbx'):
            # Get last directory name from root
            last_subdir = os.path.basename(root)
            # Combine last subdir and filename
            partial_path = f"{last_subdir}/{file}"
            raws.append(f"{base_raw_url}/{partial_path}")


In [ ]:
!mkdir fbxs

In [ ]:
for raw_url in raws:
    file_name = os.path.basename(raw_url)
    !wget -q -O fbxs/{file_name} {raw_url}

In [ ]:
import shutil
!zip -r fbxs.zip fbxs

In [ ]:
zip_file_path = 'fbxs.zip'

# Blender

In [ ]:
import os
import inspect
from IPython.display import Image

In [ ]:
# Download and Extract Blender
!wget https://download.blender.org/release/Blender4.4/blender-4.4.3-linux-x64.tar.xz  -O /content/BlenderDownload.tar.xz
#!wget https://download.blender.org/release/Blender4.4/blender-4.4.0-linux-x64.tar.xz -O /content/BlenderDownload.tar.xz
!tar -xf /content/BlenderDownload.tar.xz -C /content/

In [ ]:
# Download FBX model
#!wget -O /content/model.fbx "https://github.com/xrtlab/Validated-Avatar-Library-for-Inclusion-and-Diversity---VALID/raw/refs/heads/main/Avatars/Asian/Asian_M_2_Casual.fbx"
blender_path = 'blender-4.4.3-linux-x64/blender'

In [ ]:
%%writefile /content/fbx_to_blend.py
import bpy

# Clear existing scene
bpy.ops.wm.read_factory_settings(use_empty=True)

# Path to your FBX file
fbx_path = "/content/model.fbx"

# Import FBX
bpy.ops.import_scene.fbx(filepath=fbx_path)

# (Optional) Save the result
bpy.ops.wm.save_as_mainfile(filepath="/content/model.blend")

In [ ]:
# Run Blender in background mode with the import script
#!./{blender_path} -b --python /content/fbx_to_blend.py

In [ ]:
def fbx_to_blend():
  function_name = inspect.currentframe().f_code.co_name
  blender_cmd = f"./{blender_path} -b --python /content/{function_name}.py"
  os.system(blender_cmd)

In [ ]:
!mkdir blends

In [ ]:
import os
import shutil

fbx_folder = '/content/fbxs'
temp_fbx_path = '/content/model.fbx'
blend_folder = '/content/blends'

# Loop through files in the 'fbxs' folder
for filename in os.listdir(fbx_folder):
    if filename.endswith(".fbx"):
        source_path = os.path.join(fbx_folder, filename)
        # Copy the FBX file to '/content/model.fbx'
        shutil.copyfile(source_path, temp_fbx_path)
        print(f"Copied {filename} to {temp_fbx_path}")
        fbx_to_blend()
        print(f"Processed {filename}")
        base, _ = os.path.splitext(filename)   # split name and extension
        new_filename = base + ".blend"
        shutil.copyfile('/content/model.blend', os.path.join(blend_folder,new_filename))

In [ ]:
%%writefile blend_to_png_model.py
import bpy
scene = bpy.context.scene
scene.render.engine = 'CYCLES'
scene.render.filepath = '/content/model.png'
scene.render.image_settings.file_format = 'PNG'

scene.cycles.samples = 8
scene.cycles.use_adaptive_sampling = True
scene.cycles.use_denoising = True
scene.render.resolution_x = 800  #800
scene.render.resolution_y = 600  #600
scene.render.resolution_percentage = 100

# Create a camera if one doesn't exist

cam_data = bpy.data.cameras.new(name="Camera")
cam = bpy.data.objects.new("Camera", cam_data)
bpy.context.collection.objects.link(cam)
scene.camera = cam

cam.rotation_euler = (0, 0, 0)
cam.location = (0, -5, 1.6)
cam.rotation_euler = (1.5708, 0, 0)

sun_data = bpy.data.lights.new(name="Sun", type='SUN')
sun_data.energy = 3.0  # Brighter for full coverage
sun = bpy.data.objects.new(name="Sun", object_data=sun_data)
bpy.context.collection.objects.link(sun)
sun.location = (0, -2, 5)  # Above and slightly in front
sun.rotation_euler = (1.2, 0, 0)  # Pointing downward toward model


# Optional: frame the mesh (if camera is new)
bpy.ops.object.select_all(action='DESELECT')
bpy.ops.object.select_by_type(type='MESH')
bpy.ops.view3d.camera_to_view_selected()

# Render the image
bpy.ops.render.render(write_still=True)


In [ ]:
# Run Blender in background mode with the import script
#!./{blender_path} -b /content/model.blend --python /content/blend_to_png_model.py

In [ ]:
#Image("/content/model.png")

In [ ]:
def blend_to_png_model():
  function_name = inspect.currentframe().f_code.co_name
  blender_cmd = f"./{blender_path} -b /content/model.blend --python /content/{function_name}.py"
  os.system(blender_cmd)
  with open("model.png", "rb") as f:
    image_bytes = f.read()
  return image_bytes

bytes = blend_to_png_model()
Image(data=bytes)


In [ ]:
!mkdir pngs

In [ ]:
import os
import shutil

blend_folder = '/content/blends'
temp_blend_path = '/content/model.blend'
png_folder = '/content/pngs'

# Loop through files in the 'fbxs' folder
for filename in os.listdir(blend_folder):
    if filename.endswith(".blend"):
        source_path = os.path.join(blend_folder, filename)
        shutil.copyfile(source_path, temp_blend_path)
        print(f"Copied {filename} to {temp_blend_path}")
        blend_to_png_model()
        print(f"Processed {filename}")
        base, _ = os.path.splitext(filename)   # split name and extension
        new_filename = base + ".png"
        shutil.copyfile('/content/model.png', os.path.join(png_folder,new_filename))

In [ ]:
import os
import random
import shutil

def model_random_choice():
  blends_folder = '/content/blends'
  destination_blend_path = '/content/model.blend'

  # Get a list of all .blend files in the blends folder
  blend_files = [f for f in os.listdir(blends_folder) if f.endswith('.blend')]

  if blend_files:
      # Randomly choose one file
      chosen_file = random.choice(blend_files)
      source_path = os.path.join(blends_folder, chosen_file)

      # Copy the chosen file to the destination
      shutil.copyfile(source_path, destination_blend_path)
      print(f"Randomly selected and copied '{chosen_file}' to '{destination_blend_path}'")
  else:
      print(f"No .blend files found in '{blends_folder}'")

In [ ]:
model_random_choice()

In [ ]:
bytes = blend_to_png_model()
Image(data=bytes)

In [ ]:
%%writefile /content/blend_to_png_face.py
import bpy
import mathutils
import random
import math

scene = bpy.context.scene

# Get main mesh object
meshes = [obj for obj in scene.objects if obj.type == 'MESH']
main_mesh = meshes[0]

# Get bounding box corners in world coordinates
bbox_corners = [main_mesh.matrix_world @ mathutils.Vector(corner) for corner in main_mesh.bound_box]

# Find center X/Y, and max Z (top of the object)
bbox_center_xy = sum((v.xy for v in bbox_corners), mathutils.Vector((0, 0))) / 8
max_z = max(corner.z for corner in bbox_corners)
top_center = mathutils.Vector((bbox_center_xy.x, bbox_center_xy.y, max_z - 0.2))

# Define camera distance and random yaw/pitch
distance = 1.0
yaw = random.uniform(-math.radians(25), math.radians(25))   # side-to-side angle
pitch = random.uniform(-math.radians(10), math.radians(10)) # slight up/down angle

# Assume face is along negative Y axis, then rotate around Z (yaw) and X (pitch)
front_vector = mathutils.Vector((0, -1, 0))
rot = mathutils.Euler((pitch, 0, yaw), 'XYZ').to_matrix()
random_vector = rot @ front_vector

cam_location = top_center + random_vector * distance

# Create and set up camera
cam_data = bpy.data.cameras.new(name="Camera")
cam = bpy.data.objects.new("Camera", cam_data)
bpy.context.collection.objects.link(cam)
scene.camera = cam

# Point camera at top of mesh
direction = top_center - cam_location
rot_quat = direction.to_track_quat('-Z', 'Y')
cam.rotation_euler = rot_quat.to_euler()
cam.location = cam_location
cam.data.lens = 65

# Lighting
sun_data = bpy.data.lights.new(name="Sun", type='SUN')
sun = bpy.data.objects.new(name="Sun", object_data=sun_data)
bpy.context.collection.objects.link(sun)
sun_data.energy = 3.0
sun.location = (0, -2, 5)
sun.rotation_euler = (1.2, 0, 0)

# Final render settings
scene.render.engine = 'CYCLES'
scene.render.filepath = '/content/modelface.png'
scene.render.image_settings.file_format = 'PNG'
scene.render.resolution_x = 400
scene.render.resolution_y = 300
scene.render.resolution_percentage = 100
scene.cycles.samples = 8
scene.cycles.use_adaptive_sampling = True
scene.cycles.use_denoising = True

# Render
bpy.ops.render.render(write_still=True)


In [ ]:
%%writefile /content/blend_to_png_face.py
import bpy
import mathutils
import random
import math

scene = bpy.context.scene

# Get main mesh object
meshes = [obj for obj in scene.objects if obj.type == 'MESH']
main_mesh = meshes[0]

# Get bounding box corners in world coordinates
bbox_corners = [main_mesh.matrix_world @ mathutils.Vector(corner) for corner in main_mesh.bound_box]

# Find center X/Y, and max Z (top of the object)
bbox_center_xy = sum((v.xy for v in bbox_corners), mathutils.Vector((0, 0))) / 8
max_z = max(corner.z for corner in bbox_corners)
face_center = mathutils.Vector((bbox_center_xy.x, bbox_center_xy.y, max_z - 0.2))

# --- Camera position ---
# Randomize distance so it's not always zoomed in
distance = random.uniform(1.2, 2.5)

# Wider yaw/pitch range for more variation
yaw = random.uniform(-math.radians(60), math.radians(60))   # side-to-side
pitch = random.uniform(-math.radians(25), math.radians(25)) # up/down

# Base front direction is -Y
front_vector = mathutils.Vector((0, -1, 0))
rot = mathutils.Euler((pitch, 0, yaw), 'XYZ').to_matrix()
random_vector = rot @ front_vector

cam_location = face_center + random_vector * distance

# --- Camera setup ---
cam_data = bpy.data.cameras.new(name="Camera")
cam = bpy.data.objects.new("Camera", cam_data)
bpy.context.collection.objects.link(cam)
scene.camera = cam

# Point camera at face center
direction = face_center - cam_location
rot_quat = direction.to_track_quat('-Z', 'Y')
cam.rotation_euler = rot_quat.to_euler()
cam.location = cam_location
cam.data.lens = 65

# --- Lighting ---
sun_data = bpy.data.lights.new(name="Sun", type='SUN')
sun = bpy.data.objects.new(name="Sun", object_data=sun_data)
bpy.context.collection.objects.link(sun)
sun_data.energy = 3.0
sun.location = (0, -2, 5)
sun.rotation_euler = (1.2, 0, 0)

# --- Render settings ---
scene.render.engine = 'CYCLES'
scene.render.filepath = '/content/modelface.png'
scene.render.image_settings.file_format = 'PNG'
scene.render.resolution_x = 600
scene.render.resolution_y = 600
scene.render.resolution_percentage = 100
scene.cycles.samples = 16
scene.cycles.use_adaptive_sampling = True
scene.cycles.use_denoising = True

# Render
bpy.ops.render.render(write_still=True)


In [ ]:
#!./{blender_path} -b /content/model.blend --python /content/blend_to_png_face.py

In [ ]:
import os, inspect
import matplotlib.pyplot as plt
from IPython.display import Image
from PIL import Image as PILImage
import io

def blend_to_png_face():
    function_name = inspect.currentframe().f_code.co_name
    blender_cmd = f"./{blender_path} -b /content/model.blend --python /content/{function_name}.py"
    os.system(blender_cmd)
    with open("modelface.png", "rb") as f:
        image_bytes = f.read()
    return image_bytes


Image(data=blend_to_png_face())

In [ ]:
# Generate random images
def generate_random_images(num_images):
  images = []
  for _ in range(4):
      img_bytes = blend_to_png_face()
      pil_img = PILImage.open(io.BytesIO(img_bytes))
      images.append(pil_img)

  # --- Display in grid ---
  rows, cols = 2, 2
  fig, axs = plt.subplots(rows, cols, figsize=(8, 8))

  for ax, img in zip(axs.flat, images):
      ax.imshow(img)
      ax.axis("off")

  # If fewer images than grid slots, hide extra axes
  for ax in axs.flat[len(images):]:
      ax.axis("off")

  plt.tight_layout()
  plt.show()

In [ ]:
import math
import io
from PIL import Image as PILImage
import matplotlib.pyplot as plt

# Generate random images
def generate_random_images(num_images):
    images = []
    for _ in range(num_images):
        img_bytes = blend_to_png_face()
        pil_img = PILImage.open(io.BytesIO(img_bytes))
        images.append(pil_img)

    # --- Compute grid size automatically ---
    cols = math.ceil(math.sqrt(num_images))
    rows = math.ceil(num_images / cols)

    fig, axs = plt.subplots(rows, cols, figsize=(3*cols, 3*rows))

    # axs could be 1D if rows==1 or cols==1
    axs = axs.flatten() if hasattr(axs, 'flatten') else [axs]

    for ax, img in zip(axs, images):
        ax.imshow(img)
        ax.axis("off")

    # Hide unused axes if grid is larger than number of images
    for ax in axs[len(images):]:
        ax.axis("off")

    plt.tight_layout()
    plt.show()


In [ ]:
generate_random_images(6)

In [ ]:
model_random_choice()
generate_random_images(3)